# Setting up the Notebook

KBMOD uses Python's logging library for output during the run. If you are interested in running with debug output (verbose mode), you can set the level of Python's logger to `WARNING` (for warning messages only), `INFO` (for moderate output), or `DEBUG` (for comprehensive output). By default logging is set to warning level. We set it to DEBUG here to see **EVERYTHING** that is happening.

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

# Load the data.

We load the data using the `ImageCollection` class to load all the FITS files in the given directory.

`KBMODV0_5Config` provides some parameters about how to read the files. The important ones for this investiation are the ones involved with masking:

* bit_flag_map is a dictionary from masking reason to the bit set in the mask layer, such as: `{ "BAD": 2**0, "EDGE": 2**4, "NO_DATA": 2**8, "UNMASKEDNAN": 2**15 }`
* mask_flags is a list of which reasons to mask such as `["BAD", "UNMASKEDNAN"]`
    
The combination of those two flags will mask all pixels with the 1st and/or 15th bits set in the mask layer.

Below we are not masking anything because I don't know which bits correspond to which flags for this survery.

In [1]:
filepath = "/astro/users/jkubica/dwarf_star_data"

from kbmod.image_collection import ImageCollection
from kbmod.standardizers.fits_standardizers.kbmodv05 import KBMODV0_5, KBMODV0_5Config


load_config = KBMODV0_5Config(mask_flags=[])

# Loading from the files takes a while (multiple minutes).
ic = ImageCollection.fromDir(filepath, force=KBMODV0_5, config=load_config)
wu = ic.toWorkUnit()
wu.print_stats()

Image 0 has 86.42281308487213 percent masked pixels.
Image 0 has invalid variance values: -0.017730988562107086 < 1e-20
Image 1 has 51.92470909658097 percent masked pixels.
Image 6 has 52.000097937241264 percent masked pixels.
Image 7 has invalid variance values: -0.010657785460352898 < 1e-20
Image 9 has invalid variance values: -0.011663933284580708 < 1e-20
Image 12 has 86.4467341291876 percent masked pixels.
Image 12 has invalid variance values: -0.01934119127690792 < 1e-20
Image 13 has 60.83735326381565 percent masked pixels.
Image 13 has invalid variance values: -0.008681497536599636 < 1e-20
Image 22 has 56.63001758683907 percent masked pixels.
Image 27 has invalid variance values: -0.021550634875893593 < 1e-20


WorkUnit:
  Num Constituent Images (29):
  Reprojected: False
Image Stack Statistics:
  Image Count: 29
  Image Size: 6295 x 6261 = 39412995
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|  idx |     Time   |  Flux Min  |  Flux Max  |  Flux Mean |  Var Min |  Var Max | Var Mean | Masked |
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|    0 |  55677.544 |    -129.07 |   50170.41 |       2.64 |    -0.02 |     0.03 |     0.01 |  86.42 |
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|    1 |  56404.531 |    -154.93 |   57970.41 |       2.30 |     0.00 |     0.02 |     0.01 |  51.92 |
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|    2 |  54985.429 |    -285.37 |  109745.98 |       2.88 |     0.00 |     0.00 |     0.00 |  24.04 |
+------+------------+------------+-

# Set up the KBMOD Search

The standard approach to running KBMOD is to perform a grid search over all starting pixels and a grid of velocities. Let’s do a grid search with:
* 21 different velocity steps from -2 pixels per day and 2 pixels per day in the x dimension
* 21 different velocity steps from -2 pixels per day and 2 pixels per day in the y dimension

KBMOD needs a series of configuration parameters to specify all the information about the search. For the full list of parameters see:

https://epyc.astro.washington.edu/~kbmod/user_manual/search_params.html

In this notebook we explicitly provide the configuration parameters as a dictionary so users can see what is being specified. However most users will want to use the `SearchConfiguration` class. A `SearchConfiguration` object uses reasonable defaults when created:

In [2]:
from kbmod.configuration import SearchConfiguration

input_parameters = {
    # Use search parameters (including a force ecliptic angle of 0.0)
    # to match what we know is in the demo data.
    "generator_config": {
        "name": "VelocityGridSearch",
        "vx_steps": 21,
        "min_vx": -2.0,
        "max_vx": 2.0,
        "vy_steps": 21,
        "min_vy": -2.0,
        "max_vy": 2.0,
    },
    # Output parameters
    "result_filename": "./results.ecsv",
    # Basic filtering (always applied)
    "num_obs": 15,  # <-- Filter anything with fewer than 15 observations
    "lh_level": 10.0,  # <-- Filter anything with a likelihood < 10.0
    # SigmaG clipping parameters
    "sigmaG_lims": [25, 75],  # <-- Clipping parameters (lower and upper percentile)
    # Other parameters
    "cpu_only": True,  # <-- This will be absurdly slow.  Set to False if you have a good enough GPU.
}
config = SearchConfiguration.from_dict(input_parameters)

# Make the WorkUnit use this configuration.
wu.config = config

In [ ]:
from kbmod.run_search import SearchRunner

rs = SearchRunner()
results = rs.run_search_from_work_unit(wu)

INFO:kbmod.trajectory_generator:Creating trajectory generator of type VelocityGridSearch
DEBUG:kbmod.trajectory_generator:{'name': 'VelocityGridSearch', 'vx_steps': 21, 'min_vx': -2.0, 'max_vx': 2.0, 'vy_steps': 21, 'min_vy': -2.0, 'max_vy': 2.0}
DEBUG:kbmod.search.run_search:Starting preparing Psi and Phi images timer.
INFO:kbmod.search.psi_phi_array:Building 58 temporary 6295 by 6261 images, requiring 9143814840 bytes.
DEBUG:kbmod.search.psi_phi_array:Allocating CPU memory for float PsiPhi array using 9143814840 bytes
DEBUG:kbmod.search.psi_phi_array:Allocating times on CPU: 232 bytes.
DEBUG:kbmod.search.run_search:Finished preparing Psi and Phi images in 14.339000 seconds.
DEBUG:kbmod.run_search:VelocityGridSearch:
    Vel X: [-2.0, 2.0] in 21 steps.
    Vel Y: [-2.0, 2.0] in 21 steps.
DEBUG:kbmod.search.run_search:Starting Running batch search timer.
INFO:kbmod.search.run_search:Searching X=[0, 6261] Y=[0, 6295]
Allocating space for 315303960 results.
